## Week 8 Lab Instructions:



In [8]:
import postgresql
import psycopg2
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Data location:
# https://archive.ics.uci.edu/ml/machine-learning-databases/adult/

# DF example:
# df = pandas.read_csv('C:\\Users\\Travis Millburn\\OneDrive - University of New Haven\\Class Materials\\Spring 2023\\Week 8\\US Census Data\\census_data.csv')

### 1. Return to the US Census Income Dataset from previous weeks
    * We are very familiar with this data at this point.
    * Instantiate the DataFrame in Pandas

In [9]:
conn = psycopg2.connect(user='new_haven_ds_read', 
                        password='new_haven_ds_secret_99',
                        host='nhds-fall-23.cwroivw0q1rc.us-east-1.rds.amazonaws.com',
                        database='nhds')

In [10]:
sql = 'select * from nhds.uci_adults'

In [11]:
cursor = conn.cursor()

In [12]:
cursor.execute(sql)

In [13]:
result = cursor.fetchall()

In [14]:
cols = [desc[0] for desc in cursor.description]
cols

['age',
 'workclass',
 'fnlwgt',
 'education',
 'educational_num',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'gender',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'income',
 'id']

In [15]:
df = pd.DataFrame(result,columns=cols)
df.tail()

,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income,id
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K,32557
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K,32558
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K,32559
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K,32560
32560,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,>50K,32561


In [16]:
df.dtypes

age                 int64
workclass          object
fnlwgt              int64
education          object
educational_num     int64
marital_status     object
occupation         object
relationship       object
race               object
gender             object
capital_gain        int64
capital_loss        int64
hours_per_week      int64
native_country     object
income             object
id                  int64
dtype: object

In [17]:
for i in df.columns:
    if df[i].dtype == type(object):
        encoder = preprocessing.LabelEncoder()
        df[i] = encoder.fit_transform(df[i])
        

In [18]:
df.dtypes

age                int64
workclass          int64
fnlwgt             int64
education          int64
educational_num    int64
marital_status     int64
occupation         int64
relationship       int64
race               int64
gender             int64
capital_gain       int64
capital_loss       int64
hours_per_week     int64
native_country     int64
income             int64
id                 int64
dtype: object

In [19]:
df = df.dropna()

In [20]:
df.tail()

,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income,id
32556,27,4,257302,7,12,2,13,5,4,0,0,0,38,39,0,32557
32557,40,4,154374,11,9,2,7,0,4,1,0,0,40,39,1,32558
32558,58,4,151910,11,9,6,1,4,4,0,0,0,40,39,0,32559
32559,22,4,201490,11,9,4,1,3,4,1,0,0,20,39,0,32560
32560,52,5,287927,11,9,2,4,5,4,0,15024,0,40,39,1,32561


### 2. Split into training/testing sets

In [21]:
X = df.drop(columns='income')
y = df[['income']]

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### 3. Build a DecisionTreeClassifier with 5 fold cross validation   
    * Note results 

In [23]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
res = sklearn.model_selection.cross_val_score(clf, X_train,y_train, cv=5)
print(res)

[0.8042588  0.80671581 0.80118755 0.8042588  0.80548731]


In [27]:
print(res.mean(), res.std(), res.max())
z=res.mean()

0.8043816543816543 0.0018386522785130025 0.8067158067158067


### 4. Using the bagging technique, build an ensemble with 100 base DecisionTreeClassifier trees
    * Note and compare results 

In [28]:
%%time
# BAGGING
# Create a bag of estimators of size 100
dt_bag = BaggingClassifier(base_estimator=clf, n_estimators=1000, n_jobs=4)

# Fit / Train model
dt_bag.fit(X_train,y_train)

#Results
Bagging = dt_bag.score(X_test, y_test)
Bagging

/Users/payal/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


CPU times: user 1.02 s, sys: 1.37 s, total: 2.39 s
Wall time: 43.4 s


0.8636531138680752

### 5. Build a RandomForestClassifier   
    * Note and compare results yet again

In [25]:
from sklearn.ensemble import RandomForestClassifier

random_forest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

random_forest_classifier.fit(X_train, y_train.values.ravel())

random_forest_accuracy = random_forest_classifier.score(X_test, y_test)
print("Random Forest Accuracy:", random_forest_accuracy)

Random Forest Accuracy: 0.8656184743888957


### 6. Lastly, build a different model of your choice.  Compare the results to the best model of the three (rf, bagging, or cv) models built in 1-5.   

In [26]:
from sklearn.linear_model import LogisticRegression

logistic_regression_classifier = LogisticRegression(random_state=42)

logistic_regression_classifier.fit(X_train, y_train.values.ravel())

logistic_regression_accuracy = logistic_regression_classifier.score(X_test, y_test)
print("Logistic Regression Accuracy:", logistic_regression_accuracy)


Logistic Regression Accuracy: 0.798182041518241


/Users/payal/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [34]:
results_final = pd.DataFrame(
    [[res.mean(), Bagging, random_forest_accuracy, logistic_regression_accuracy]],
    index=['Accuracy'],
    columns=['Cross-Validation', 'Bagging', 'Random Forest', 'Logistic Regression']
)

print(results_final)

          Cross-Validation   Bagging  Random Forest  Logistic Regression
Accuracy          0.804382  0.863653       0.865618             0.798182
